In [6]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio


# Force Jupyter rendering (choose one depending on environment)
pio.renderers.default = "notebook"  # or "iframe_connected" for JupyterLab

df = pd.read_csv('dataset.csv')

fig = px.bar(
    industry_counts,
    x="Industry",
    y="Company Count",
    color="Country",
    barmode="group",
    title="Number of Companies per Industry and Country",
    template="plotly_white"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Define your desired segment order
segment_order = ["Large Corporate", "Medium Corporate", "Small Corporate"]

# Ensure Segment column is treated as ordered categorical
df["Segment"] = pd.Categorical(df["Segment"], categories=segment_order, ordered=True)

# Loop through each segment in the specified order
for segment in segment_order:
    df_segment = df[df["Segment"] == segment]

    # Sort industries by median CO₂ for this segment
    industry_order = (
        df_segment.groupby("Industry")["CO2 (tons)"]
        .median()
        .sort_values(ascending=False)
        .index.tolist()
    )

    # Create box plot
    fig = px.box(
        df_segment,
        x="Industry",
        y="CO2 (tons)",
        color="Industry",
        title=f"CO₂ Emissions by Industry – Segment: {segment}",
        labels={"CO2 (tons)": "CO₂ (tons)"},
        category_orders={
            "Industry": industry_order
        }
    )

    fig.update_layout(
        xaxis_tickangle=-45,
        showlegend=False
    )

    fig.show()

# Create scatter plot
fig = px.scatter(
    df,
    x="Financed Amount",
    y="CO2 (tons)",
    color="Industry",           # Or use Segment
    hover_data=["Country", "Segment", "Industry"],
    title="CO₂ vs Financed Amount by Industry",
    labels={
        "Financed Amount": "Financed Amount",
        "CO2 (tons)": "CO₂ Emissions (tons)",
        "Water Usage (thousand m3)": "Water Usage (000 m³)"
    },
    size_max=25
)

fig.update_layout(
    template="plotly_white"
)

fig.show()

#Group by country and calculate average risk values
risk_avg = df.groupby("Country")[["Flood Risk", "Water Stress", "Drought Risk"]].mean().reset_index()

# Melt to long format for heatmap
risk_melted = risk_avg.melt(
    id_vars="Country",
    value_vars=["Flood Risk", "Water Stress", "Drought Risk"],
    var_name="Risk Type",
    value_name="Average Risk"
)

# Round average risk values for cleaner labels
risk_melted["Average Risk"] = risk_melted["Average Risk"].round(2)


fig = px.density_heatmap(
    risk_melted,
    x="Risk Type",
    y="Country",
    z="Average Risk",
    color_continuous_scale="YlGnBu",
    text_auto=True,
    title="Environmental Risk Averages by Country",
    labels={"Average Risk": "Avg Risk Score"},
    template="plotly_white"
)

fig.update_layout(
    height=600,
    coloraxis_colorbar=dict(title=None)  # Remove colorbar title
)

fig.show()

fig = px.scatter(
    df,
    x="Water Stress",
    y="Water Usage (thousand m3)",
    color="Country",
    hover_data=["Industry", "Segment", "Financed Amount"],
    title="💧 Water Usage vs. Water Stress by Country",
    labels={
        "Water Stress": "Water Stress Score",
        "Water Usage (thousand m3)": "Water Usage (thousand m³)"
    },
    template="plotly_white"
)

fig.update_traces(marker=dict(size=8, opacity=0.6))
fig.update_layout(height=600)
fig.show()

In [87]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load your data
df = pd.read_csv("dataset.csv")

# Preprocess data for bar chart
industry_counts = df.groupby(["Industry", "Country"]).size().reset_index(name="Company Count")

# Preprocess segment order
df["Segment"] = pd.Categorical(df["Segment"], categories=["Large Corporate", "Medium Corporate", "Small Corporate"], ordered=True)

# Heatmap risk data
risk_avg = df.groupby("Country")[["Flood Risk", "Water Stress", "Drought Risk"]].mean().reset_index()
risk_melted = risk_avg.melt(
    id_vars="Country",
    value_vars=["Flood Risk", "Water Stress", "Drought Risk"],
    var_name="Risk Type",
    value_name="Average Risk"
)
risk_melted["Average Risk"] = risk_melted["Average Risk"].round(2)

# Plot 1: Bar chart
fig_bar = px.bar(
    industry_counts,
    x="Industry",
    y="Company Count",
    color="Country",
    barmode="group",
    title="Number of Companies per Industry and Country",
    template="plotly_white"
)
fig_bar.update_layout(xaxis_tickangle=-45)

# Plot 2: CO2 vs Financed Amount
fig_scatter = px.scatter(
    df,
    x="Financed Amount",
    y="CO2 (tons)",
    color="Industry",
    hover_data=["Country", "Segment", "Industry"],
    title="CO₂ vs Financed Amount by Industry",
    labels={"CO2 (tons)": "CO₂ Emissions (tons)"},
    template="plotly_white"
)
fig_scatter.update_layout(height=500)

# Plot 3: Risk heatmap
fig_risk = px.density_heatmap(
    risk_melted,
    x="Risk Type",
    y="Country",
    z="Average Risk",
    color_continuous_scale="YlGnBu",
    text_auto=True,
    title="Environmental Risk Averages by Country",
    labels={"Average Risk": "Avg Risk Score"},
    template="plotly_white"
)
fig_risk.update_layout(height=600, coloraxis_colorbar=dict(title=None))

# Plot 4: Water Usage vs Water Stress
fig_water = px.scatter(
    df,
    x="Water Stress",
    y="Water Usage (thousand m3)",
    color="Country",
    hover_data=["Industry", "Segment", "Financed Amount"],
    title="Water Usage vs. Water Stress by Country",
    labels={"Water Usage (thousand m3)": "Water Usage (000 m³)"},
    template="plotly_white"
)
fig_water.update_traces(marker=dict(size=8, opacity=0.6))
fig_water.update_layout(height=600)

# Dash app
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Environmental & Financial Insights Dashboard", style={"textAlign": "center"}),

    dcc.Graph(figure=fig_bar),
    dcc.Graph(figure=fig_scatter),
    dcc.Graph(figure=fig_risk),
    dcc.Graph(figure=fig_water),

    html.H2("CO₂ Emissions by Industry – Split by Segment"),
    *[
        html.Div([
            html.H4(f"Segment: {segment}"),
            dcc.Graph(
                figure=px.box(
                    df[df["Segment"] == segment],
                    x="Industry",
                    y="CO2 (tons)",
                    color="Industry",
                    title=f"CO₂ Emissions by Industry – Segment: {segment}",
                    category_orders={
                        "Industry": df[df["Segment"] == segment].groupby("Industry")["CO2 (tons)"].median().sort_values(ascending=False).index.tolist()
                    },
                    labels={"CO2 (tons)": "CO₂ (tons)"},
                    template="plotly_white"
                ).update_layout(showlegend=False, xaxis_tickangle=-45)
            )
        ])
        for segment in ["Large Corporate", "Medium Corporate", "Small Corporate"]
    ]
])

if __name__ == "__main__":
    app.run_server(debug=True)


ObsoleteAttributeException: app.run_server has been replaced by app.run